In [3]:
import os
import pandas as pd

# Loading files

In [4]:
# Example usage
FILES_PATH = 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/'

In [5]:
def list_files(data_path):
    return os.listdir(data_path)

def list_files_with_extension(data_path, extension):
    return [f for f in os.listdir(data_path) if f.endswith(extension)]

def list_files_with_extension_directory(data_path, extension):  
    return [FILES_PATH + f for f in os.listdir(data_path) if f.endswith(extension)]
# "FileName": os.path.basename(pdf) # get the file name without the path

In [6]:
txt_files_directory = list_files_with_extension_directory(FILES_PATH, '.txt')
txt_files_directory

['Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay001.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay002.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay003.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay004.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay005.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay006.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay007.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay008.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay009.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay010.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay011.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay012.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay013.txt',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay014.txt',
 'Essa

In [7]:
example_txt = txt_files_directory[0]
example_txt

'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay001.txt'

In [8]:
# load the text file
with open(example_txt, 'r') as file:
    text = file.read()
print(text)

Should students be taught to compete or to cooperate?

It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.
First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students. What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others. During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think compr

In [9]:
ann_files_directory = list_files_with_extension_directory(FILES_PATH, '.ann')
ann_files_directory

['Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay001.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay002.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay003.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay004.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay005.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay006.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay007.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay008.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay009.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay010.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay011.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay012.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay013.ann',
 'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay014.ann',
 'Essa

In [10]:
ann_files_directory[0]

'Essays/ArgumentAnnotatedEssays-2.0/brat-project-final/essay001.ann'

In [36]:
print(f"Anzahl Text-Dateien: {len(txt_files_directory)}")
print(f"Anzahl Brat-Dateien: {len(ann_files_directory)}")

Anzahl Text-Dateien: 402
Anzahl Brat-Dateien: 402


In [42]:
# create dataframe with file names
df = pd.DataFrame()
df['txt_path'] = txt_files_directory
df['ann_path'] = ann_files_directory
df['txt'] = df['txt_path'].apply(lambda x: os.path.basename(x))
df['ann'] = df['ann_path'].apply(lambda x: os.path.basename(x))
df.head()

,txt_path,ann_path,txt,ann
0,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,essay001.txt,essay001.ann
1,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,essay002.txt,essay002.ann
2,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,essay003.txt,essay003.ann
3,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,essay004.txt,essay004.ann
4,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,Essays/ArgumentAnnotatedEssays-2.0/brat-projec...,essay005.txt,essay005.ann


In [ ]:
# Train-Test-Split
from sklearn.model_selection import train_test_split

# Split the dataframe into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the first few rows of the training and test sets
print(f"Training DataFrame: {train_df.shape}")
print(f"\nTest DataFrame: {test_df.shape}")

Training DataFrame: (321, 4)

Test DataFrame: (81, 4)


# loading the model

In [11]:
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint

In [12]:
# get the API key from the .env file
load_dotenv()
llama_api = os.getenv("HUGGINGFACE_TOKEN")

In [13]:
model_id = "meta-llama/Llama-3.2-3B-Instruct"

llm = HuggingFaceEndpoint(repo_id=model_id,
                          huggingfacehub_api_token=llama_api)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\ben-s\.cache\huggingface\token
Login successful


# Prompt Template

In [14]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

In [28]:
PROMPT_PATH = "prompts"
list_files(PROMPT_PATH)

['chain-of-thought.txt', 'role_play.txt', 'zero-shot.txt']

In [ ]:
# load the text files with the prompts
prompt_files = list_files_with_extension(PROMPT_PATH, '.txt')
prompt_files

# load the first prompt file
with open(f"{PROMPT_PATH}/{prompt_files[0]}", 'r') as file:
    prompt = file.read()
print(prompt)

# load the second prompt file
with open(f"{PROMPT_PATH}/{prompt_files[1]}", 'r') as file:
    prompt = file.read()


In [22]:
# load the text file
with open(PROMPT_PATH + '/zero-shot.txt', 'r') as file:
    zero_shot = file.read()
print(zero_shot)

You are an expert in argument mining. Extract the argumentative units “major claim”, “claim”, and “premise” from a text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim. It is possible that there are several major claims. 
Return only the argumentative units and relationships in the following structured way:
## Major Claims:
-MC1:
-MC2:
...
## Claims:
-C1:
-C2:
...
## Premises:
-P1:
-P2:
...
## Argumentative relations:
-C1 for MC
-C2 against MC
-P1 supports C1
-P2 attacks C2
...


In [ ]:
with open(PROMPT_PATH + '/chain-of-thought.txt', 'r') as file:
    cot = file.read()
print(cot)

You are an expert in argument mining and argument extraction from texts. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from a text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim. It is possible that there are several major claims. Provide a step-by-step solution. Return the argumentative units and relationships in the following structured way:
# Argument Mining
## Major Claims:
-MC1:
-MC2:
...
## Claims:
-C1:
-C2:
...
## Premises:
-P1:
-P2:
...
## Argumentative relations:
-C1 for MC
-C2 against MC
-P1 supports C1
-P2 attacks C2
...


In [32]:
with open(PROMPT_PATH + '/role_play.txt', 'r') as file:
    role_play = file.read()
print(role_play)

You are an expert in argument mining and argument extraction from texts. Extract the argumentative units “major claim”, “claim”, and “premise” as parts from a text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim. It is possible that there are several major claims. Return only the argumentative units and relationships in the following structured way:
# Argument Mining
## Major Claims:
-MC1:
-MC2:
...
## Claims:
-C1:
-C2:
...
## Premises:
-P1:
-P2:
...
## Argumentative relations:
-C1 for MC
-C2 against MC
-P1 supports C1
-P2 attacks C2
...


In [27]:
# get data type of zero_shot
type(zero_shot)

str

In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{system_message}"),
        ("user", "{argument_text}"),
    ]
)


print(
    prompt_template.invoke(
        {
            "system_message": zero_shot,
            "argument_text": text,
        }
    )
)


messages=[SystemMessage(content='You are an expert in argument mining. Extract the argumentative units “major claim”, “claim”, and “premise” from a text. Also extract the argumentative relationships between the units. Claims can be “for” or “against” the major claims. Premises, on the other hand, can “support” or “attack” a claim. It is possible that there are several major claims. \nReturn only the argumentative units and relationships in the following structured way:\n## Major Claims:\n-MC1:\n-MC2:\n...\n## Claims:\n-C1:\n-C2:\n...\n## Premises:\n-P1:\n-P2:\n...\n## Argumentative relations:\n-C1 for MC\n-C2 against MC\n-P1 supports C1\n-P2 attacks C2\n...', additional_kwargs={}, response_metadata={}), HumanMessage(content="Should students be taught to compete or to cooperate?\n\nIt is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the wh

# Combine llm and prompt template

In [ ]:
# combine the prompt template and the llm
llm_chain = prompt_template | llm


argument_text = text

answer = llm_chain.invoke({"system_message": cot,
                           "argument_text": argument_text})
print(answer)

 This implies that we should attach more importance to cooperation during primary education.
# Argument Mining
## Major Claims:
-MC1: We should attach more importance to cooperation during primary education.
-MC2: No matter from the view of individual development or the relationship between competition and cooperation, a more cooperative attitude is more profitable in one's success.
## Claims:
-C1: We should attach more importance to cooperation during primary education.
-C2: A more cooperative attitude is more profitable in one's success.
## Premises:
-P1: Through cooperation, children can learn about interpersonal skills which are significant in the future life of all students.
-P2: The significance of competition is that how to become more excellence to gain the victory.
-P3: The greater our goal is, the more competition we need.
-P4: It is hard to imagine how an athlete could win the game without the training of his or her coach, and the help of other professional staffs such as th